In [1]:
import pandas as pd
import pyodbc
import sqlite3

# iedereen heeft andere servername btw

In [2]:
servername = 'LAPTOP-RPEMJO16\\SQLEXPRESS'

# Northwind connectie

In [3]:
northwind_conn = pyodbc.connect('DRIVER={SQL SERVER};SERVER='+servername+';DATABASE=NorthWind; Trusted_Connection=yes')
nwCursor = northwind_conn.cursor()
nwCursor

# Adventureworks connectie

In [4]:
adventureworks_conn = pyodbc.connect('DRIVER={SQL SERVER};SERVER='+servername+';DATABASE=AdventureWorks2019; Trusted_Connection=yes')
awCursor = adventureworks_conn.cursor()
awCursor

# Aenc connectie

In [5]:
aenc_conn = pyodbc.connect('DRIVER={Microsoft Access Driver (*.mdb, *.accdb)};DBQ=../raw-databases/aenc.accdb;')
aencCursor = aenc_conn.cursor()

['MSysAccessStorage', 'MSysACEs', 'MSysComplexColumns', 'MSysIMEXColumns', 'MSysIMEXSpecs', 'MSysNameMap', 'MSysNavPaneGroupCategories', 'MSysNavPaneGroups', 'MSysNavPaneGroupToObjects', 'MSysNavPaneObjectIDs', 'MSysObjects', 'MSysQueries', 'MSysRelationships', 'MSysResources', 'bonus', 'customer', 'department', 'employee', 'product', 'region', 'sales_order', 'sales_order_item', 'state']


In [6]:
aenctest = pd.read_sql_query('SELECT * FROM customer', aenc_conn)
awtest = pd.read_sql_query('SELECT * FROM Person.Address', adventureworks_conn)
nwtest = pd.read_sql_query('SELECT * FROM customers', northwind_conn)

C:\Users\kwand\AppData\Local\Temp\ipykernel_15844\2905875312.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  aenctest = pd.read_sql_query('SELECT * FROM customer', aenc_conn)
C:\Users\kwand\AppData\Local\Temp\ipykernel_15844\2905875312.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  awtest = pd.read_sql_query('SELECT * FROM Person.Address', adventureworks_conn)
C:\Users\kwand\AppData\Local\Temp\ipykernel_15844\2905875312.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  nwtest = pd.read_sql_query('SELECT * FROM cu

In [61]:
# Klopt echt helemaal niets van de aenc - > nw orders draw io

# Select NorthWind
nw_Emp = pd.read_sql_query('SELECT * FROM Employees', northwind_conn)
nw_Ord = pd.read_sql_query('SELECT * FROM Orders', northwind_conn)
nw_OrderDetails = pd.read_sql_query('SELECT * FROM [Order Details]', northwind_conn)
# Select Adventureworks
Prod_Prod = pd.read_sql_query('SELECT * FROM Production.Product', adventureworks_conn)
Prod_Inv = pd.read_sql_query('SELECT * FROM Production.ProductInventory', adventureworks_conn)

Sales_Sales = pd.read_sql_query('SELECT * FROM Sales.SalesOrderDetail', adventureworks_conn)
Sales_OrderHeader = pd.read_sql_query('SELECT * FROM Sales.SalesOrderHeader', adventureworks_conn)


# Select aenc
AE_Customer = pd.read_sql_query('SELECT * FROM customer', aenc_conn)
AE_SalesOrderItem = pd.read_sql_query('SELECT * FROM sales_order_item', aenc_conn)
AE_SalesOrder = pd.read_sql_query('SELECT * FROM sales_order', aenc_conn)
AE_SalesOrderMerged = AE_SalesOrder.merge(AE_SalesOrderItem, on='id', how='right')

AeA = pd.concat([AE_SalesOrder, AE_SalesOrderItem], axis=0)

nw_Ord2 = pd.concat([nw_Ord, AeA], axis=0)
# AENC.customer += Orders
MergeNwOrders1 = AE_Customer.merge(nw_Ord2, left_on='id', right_on='CustomerID', how='outer')
# Northwind.Employees += (AENC.customer += Orders)2\
MergeNwOrders2 = nw_Emp.merge(MergeNwOrders1, on='EmployeeID', how='right')


MergedProdProd = Prod_Inv.merge(Prod_Prod, how='outer')

NWorderDetails1 = MergeNwOrders2.merge(nw_OrderDetails, on='OrderID', how='right')
NWorderDetailsFinal = MergedProdProd.merge(NWorderDetails1, how='outer')

OrderDetails = NWorderDetailsFinal[['OrderID', 'ProductID', 'Quantity']]
#OrderDetails.dropna()
OrderDetails

C:\Users\kwand\AppData\Local\Temp\ipykernel_15844\1147426302.py:4: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  nw_Emp = pd.read_sql_query('SELECT * FROM Employees', northwind_conn)
C:\Users\kwand\AppData\Local\Temp\ipykernel_15844\1147426302.py:5: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  nw_Ord = pd.read_sql_query('SELECT * FROM Orders', northwind_conn)
C:\Users\kwand\AppData\Local\Temp\ipykernel_15844\1147426302.py:6: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  nw_OrderDetails = pd.read_sql_query('SELECT * FROM [O

0